In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
file_name = repo_path / 'data/covert-search/large-field/p3_data_AZ_t1.mat'
data = loadmat(file_name)
print(data.keys())

In [ ]:
taget_amplitude = data['targetAmplitude']
target_location = data['tLocation']
spot_center = data['spotCenters']
human_response = data['hResponse']
n_location = data['nLocations'][0][0]
spot_diameter = data['spotLength'][0][0]
stimulus_design_size= data['totalLength'][0][0]
monitor_width = data['monitorPx'][0][0]
monitor_height = data['monitorPx'][0][1]
ppd = data['ppd'][0][0]

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:,0] += (monitor_height - stimulus_design_size)//2
shifted_spot_center[:,1] += (monitor_width - stimulus_design_size)//2

accurate_response = target_location == human_response

In [ ]:
list_spot_region = [np.zeros(monitor_width,monitor_height)] * (n_location-1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location-1):
     list_spot_region[index_location] = (pixel_row - shifted_spot_center[index_location, 0])**2+(pixel_col - shifted_spot_center[index_location, 1])**2<= spot_diameter**2/4

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))

for index_location in range(n_location-1):
    stimulus_region[list_spot_region[index_location]] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(stimulus_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center')

circle = Circle((monitor_width//2, monitor_height//2), spot_diameter*1.5, edgecolor='r', facecolor='none', linewidth=3)
ax.add_patch(circle)

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

In [ ]:
eccentral_distance_sorted_indexes = np.argsort(array_eccentral_distance)
extra_eccentral_distance_sorted_indexes = np.insert(eccentral_distance_sorted_indexes+1,0,0)
print(extra_eccentral_distance_sorted_indexes)

In [ ]:
accurate_response = target_location == human_response
confusion_mat = confusion_matrix(human_response.flatten(), target_location.flatten(), labels=range(n_location))[extra_eccentral_distance_sorted_indexes, :][:, extra_eccentral_distance_sorted_indexes]
array_index_location = np.arange(n_location)[extra_eccentral_distance_sorted_indexes]

In [ ]:
correct_rejection = confusion_mat[0][0]
second_largest_confusion_mat = np.max(np.delete(confusion_mat, np.where(confusion_mat == correct_rejection)))
plotted_confusion_mat = confusion_mat.copy()
plotted_confusion_mat[0][0] = second_largest_confusion_mat
print(second_largest_confusion_mat)

In [ ]:
fig, ax = plt.subplots(figsize=(15,12))
im = ax.imshow(plotted_confusion_mat)

for index_target_location in array_index_location:
    for index_human_response in array_index_location:
        if not (-index_target_location == 0 and index_human_response == 0):
            ax.text(index_human_response, index_target_location, f'{confusion_mat[index_target_location, index_human_response]:.0f}', ha='center', va='center')

ax.annotate(f'{correct_rejection:.0f}', xy=(0, 0), xytext=(-0.5,-1), arrowprops=dict(facecolor='black', shrink=0.05))

ax.set(xticks=np.arange(n_location), yticks=np.arange(n_location), xticklabels=array_index_location, yticklabels=array_index_location, xlabel='Target location sorted by eccentral distance', ylabel='Human response sorted by eccentral distance')

inner_ax = fig.add_axes([0.25, 0.05, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[],yticks=[])

for index_location in range(n_location-1):
    inner_ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center', fontsize=16)

plt.show()

In [ ]:
indixes_near = np.arange(1,n_location)[array_eccentral_distance < spot_diameter*1.5]
indixes_far = np.arange(1,n_location)[array_eccentral_distance > spot_diameter*1.5]
print(indixes_near, indixes_far)

In [ ]:
accuracy_all = accurate_response.mean()
accuracy_near = accurate_response[np.isin(target_location, indixes_near)].mean()
accuracy_far = accurate_response[np.isin(target_location, indixes_far)].mean()
print(accuracy_all, accuracy_near, accuracy_far)

In [ ]:
local_hit_region = np.empty((monitor_height, monitor_width))

local_hit = np.array([np.logical_and(target_location == index_location+1, human_response == index_location+1).sum() / (target_location == index_location+1).sum()  for index_location in range(n_location-1)])

for index_location in range(n_location-1):
    local_hit_region[list_spot_region[index_location]] = local_hit[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_hit_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{local_hit[index_location]:.2f}', ha='center', va='center')

In [ ]:
local_miss_region = np.empty((monitor_height, monitor_width))

local_miss = np.array([np.logical_and(target_location == index_location+1, human_response == 0).sum() / (target_location == index_location+1).sum()  for index_location in range(n_location-1)])

for index_location in range(n_location-1):
    local_miss_region[list_spot_region[index_location]] = local_miss[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_miss_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{local_miss[index_location]:.2f}', ha='center', va='center')

In [ ]:
local_fa_region = np.empty((monitor_height, monitor_width))

local_fa = np.array([np.logical_and(target_location == 0, human_response == index_location+1).sum() / (target_location == 0).sum()  for index_location in range(n_location-1)])

for index_location in range(n_location-1):
    local_fa_region[list_spot_region[index_location]] = local_fa[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_fa_region)

for index_location in range(n_location-1):
    ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{local_fa[index_location]:.3f}', ha='center', va='center')

In [ ]:
cr = np.logical_and(target_location == 0, human_response == 0).sum() / (target_location == 0).sum()

print(cr)

In [ ]:
array_eccentral_distance = np.zeros((n_location-1,))

for index_location in range(n_location-1):
    array_eccentral_distance[index_location] = np.sqrt((spot_center[index_location, 0] - stimulus_design_size//2)**2+(spot_center[index_location, 1] - stimulus_design_size//2)**2)

array_eccentral_distance /= ppd

In [ ]:
orientation_hsv_color_map = np.zeros((n_location-1,3))

for index_location in range(n_location-1):
    dy = spot_center[index_location, 0] - stimulus_design_size//2
    dx = stimulus_design_size//2 - spot_center[index_location, 1]
    orientation_hsv_color_map[index_location,0] = (1 - np.arctan2(dy,dx) / np.pi) / 2

    orientation_hsv_color_map[index_location,1] = 1

    if dx == 0 and dy == 0:
        orientation_hsv_color_map[index_location,2] = 0
    else:
        orientation_hsv_color_map[index_location,2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_hit, s=250, c=rgb_orientation_color_map)

ax.set(xlabel='Eccentral distance (deg)', ylabel='Hit')

inner_ax = fig.add_axes([0.2, 0.1, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location-1):
    circle = Circle((shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_location], linewidth=1)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center', c='w', fontsize=16)

ax.axvline(x=spot_diameter*1.5/ppd, color='r', linewidth=3)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_miss, s=250, c=rgb_orientation_color_map)

ax.set(xlabel='Eccentral distance (deg)', ylabel='Miss')

inner_ax = fig.add_axes([0.2, 0.5, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location-1):
    circle = Circle((shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_location], linewidth=1)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center', c='w', fontsize=16)

ax.axvline(x=spot_diameter*1.5/ppd, color='r', linewidth=3)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_fa, s=250, c=rgb_orientation_color_map)

ax.set(xlabel='Eccentral distance (deg)', ylabel='False alarm')

inner_ax = fig.add_axes([0.2, 0.1, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location-1):
    circle = Circle((shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0]), spot_diameter/2, color=rgb_orientation_color_map[index_location], linewidth=1)
    inner_ax.add_patch(circle)
    
    inner_ax.text(shifted_spot_center[index_location, 1], shifted_spot_center[index_location, 0], f'{index_location+1}', ha='center', va='center', c='w', fontsize=16)

ax.axvline(x=spot_diameter*1.5/ppd, color='r', linewidth=3)
plt.show()

In [ ]:
local_fa